### Remarques
1. Nous sommes dans un cas paramagnétique.
2. Attention pê développement multipolaire?
3. A cause du coeur supraconducteur, le champ magnéique sera moins intense!
4. $ \mathbf{\nabla} \times \mathbf{B} = \mu \mathbf{J}$ dans le cas paramagnétique!
5. Le champ n'est pas toujours uniforme, même si ce cas semble s'apparenter à un solénoïde!
6. Voir leçons 4_3, 4_4 et pê 3_4
7. Il faut déterminer le moment où le champ B sera uniforme (condition d'arrêt du code).
8. Condition initiale pour B? B_0 = mu
9. Ne peut-on pas considérer cette brusque apparition de B comme une sorte de choc?
10. Exploiter les symétries?
11. Stabilité!! 
12. N'y a-t-il pas une durée pendant laquelle le conducteur n'est plus à l'équilibre? Donc E à l'intérieur toujours nul? Pris en compte par Faraday
13. Loi de Faraday pendant ce temps? La négliger puisque le changement semble instantané? Déjà prise en compte!

## Equation de diffusion du champ magnétique

En reprenant les équations de Maxwell suivantes, nous pouvons montrer que le champ magnétique $\mathbf{B}$ obéit à une équation de diffusion. En se rappelant la loi d'Ohm $\mathbf{J} = \sigma \mathbf{E}$,
$$\nabla \times {\bf E} = -\partial_t {\bf B}\\
\nabla \times {\bf B} =\mu {\bf J}\\
\nabla \cdot {\bf B}=0
$$
ensuite en subsituant $\mathbf{E}$ par $\dfrac{\mathbf{J}}{\sigma}$  puis $\mathbf{J}$ par $\dfrac{1}{\mu} \nabla \times {\bf B}$ dans le première équation, et en sachant que $\nabla \times(\nabla \times {\bf B}) = \nabla (\nabla \cdot {\bf B}) - \Delta \mathbf{B} = - \Delta \mathbf{B}$, nous obtenons l'équation de diffusion:
$$ \partial_t \bf{B} - \dfrac{1}{\sigma \mu} \Delta B = 0$$

## Conditions aux bords

## Evolution au cours du temps - Méthode d'Euler

In [13]:
import numpy
from matplotlib import pyplot
%matplotlib inline

Il faut implémenter le champ magnétique comme une matrice $2n$ x $2n$; le $2n$ permet de fixer la parité de la matrice. 

In [53]:
sigma = 5.96*10**7
mu = 1.26*10**-6

nt = 10
nx = 10
ny = 10
dx = 1
dy = 1
dt = 1

mx = int(nx/3)+1
my = int(ny/2)+1

B = numpy.zeros((mx,my))
B[0,:] = mu*numpy.ones(my)
B

array([[  1.26000000e-06,   1.26000000e-06,   1.26000000e-06,
          1.26000000e-06,   1.26000000e-06,   1.26000000e-06],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [57]:
def matrix_extent(B): #rend la sous-diagonale égale à la sur-diagonale
    
    for k in range(1,mx-1):
        B[k,k-1] = B[k-1,k]
    return B

In [58]:
A = matrix_extent(B)
print(A)

[[  1.26000000e-06   1.26000000e-06   1.26000000e-06   1.26000000e-06
    1.26000000e-06   1.26000000e-06]
 [  1.26000000e-06   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [59]:
def evolution(B, nt, mu,sigma, dt, dx, dy):
    
    for n in range(nt):
        B = matrix_extent(B)
        Bn = B
        
        for j in range(1,my-1):
            
            for i in range(j,mx-1):
                B[i,j] = Bn[i,j] + 1/(mu*sigma)*\
                (dt/dy**2 * (Bn[i+1,j] - 2*Bn[i,j] + Bn[i-1,j]) +\
                 dt/dx**2 * (Bn[i,j+1]- 2*Bn[i,j] + Bn[i,j-1]))
    return B

In [60]:
Bi = evolution(B, nt, mu,sigma, dt, dx, dy)